In [1]:
# Задание 1. Соединить Python с БД по API. 
import psycopg2 
import getpass
import pandas as pd
import datetime

In [9]:
# подключаемся к БД, в database указываем нужную, в которую раньше заносили таблицы и выполняли
# действия для 1 домашки, вводим свой пароль
conn=psycopg2.connect(user='postgres',
                      database='test',
                      host='localhost',
                      port='5432',
                      password=getpass.getpass('Insert password: '))

Insert password: ········


In [10]:
# Задание 2. После успешного подключения к БД запрашиваем инфо, нужную для 3 пункта.
# Запрашиваем данные из таблиц bond_description и quotes, отбираем максимальный купон в случае, 
# если встречаются разные купонные выплаты 
query = """ SELECT *
            FROM (SELECT MAX("CPN") AS "CPN", "ISIN", max("BUYBACKDATE") AS "BUYBACKDATE",
                  max("BUYBACKPRICE") AS "BUYBACKPRICE"
                  FROM bonds.quotes  
                  GROUP BY "ISIN") AS cpn
            INNER JOIN(SELECT "ISINCode", "FaceValue", "HaveOffer", "AmortisedMty", 
                       "CouponPerYear", "NumCoupons", "EndMtyDate", "CouponType", "IssuerName"
                       FROM bonds.bond_description 
                       WHERE "Status"!='Погашен') AS bonds
            ON cpn."ISIN" = bonds."ISINCode" """

In [14]:
# Далее читаем запрос в Pythonе, ISINCode можно дропнуть, чтобы не повторялся 2 раза в таблице.
data = pd.read_sql_query(query, conn)
data = pd.DataFrame(data)
data = data.drop(['ISINCode'], axis=1)
data.head(5)

,CPN,ISIN,BUYBACKDATE,BUYBACKPRICE,FaceValue,HaveOffer,AmortisedMty,CouponPerYear,NumCoupons,EndMtyDate,CouponType,IssuerName
0,64.8219,RU000A0JUPW5,2016-06-23,100.0,1000.0,True,False,2,20,2024-06-13,Переменный,Банк Зенит
1,42.3836,RU000A0JUPY1,2017-06-23,100.0,1000.0,True,False,2,20,2024-06-14,Переменный,Банк Зенит
2,79.7800,RU000A0JTPS5,None,NaN,1000.0,True,False,2,6,2016-02-25,Переменный,Запсибкомбанк
3,92.0500,RU000A0JQ664,None,NaN,1000.0,True,False,2,14,2016-06-02,Переменный,ИКС 5 ФИНАНС
4,62.3300,RU000A0JR2S7,2016-04-08,100.0,1000.0,True,False,2,14,2017-10-06,Переменный,Т Плюс
